### CONFIGURACAO AMBIENTE

In [20]:
# Importar as bibliotecas necessárias
from imblearn.over_sampling import SMOTER
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputRegressor




# Configurações do banco de dados
DB_NAME = "vitivinicultura_db"
DB_USER = "postgres"
DB_PASSWORD = "102030"
DB_HOST = "localhost"
DB_PORT = "5432"

# Criar o engine de conexão usando SQLAlchemy
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

# Criar uma conexão a partir do engine
connection = engine.raw_connection()


ImportError: cannot import name 'SMOTER' from 'imblearn.over_sampling' (/home/alberone/projects/fiap/ml-engineering/ml-tech-fiap-vitivinicultura-api/venv/lib/python3.12/site-packages/imblearn/over_sampling/__init__.py)

### DATA LOAD

In [2]:
# Carregar os dados da tabela de exportação
query = "SELECT * FROM exportacao;"
df_exportacao = pd.read_sql(query, con=connection)

# Fechar a conexão
connection.close()

# Visualizar os primeiros registros dos dados carregados
df_exportacao.head()


/tmp/ipykernel_364460/2683005586.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exportacao = pd.read_sql(query, con=connection)


,id,ano,pais,quantidade_litros,valor_usd,data_insercao
0,1,2023,Afeganistão,NaN,NaN,2025-03-30 18:27:41.917881
1,2,2023,África do Sul,117.0,698.0,2025-03-30 18:27:41.917881
2,3,2023,"Alemanha, República Democrática",4806.0,31853.0,2025-03-30 18:27:41.917881
3,4,2023,Angola,NaN,NaN,2025-03-30 18:27:41.917881
4,5,2023,Anguilla,NaN,NaN,2025-03-30 18:27:41.917881


### DATA PREPARATION

In [3]:
# Substituir "-" por NaN nas colunas 'quantidade_litros' e 'valor_usd'
df_exportacao['quantidade_litros'] = pd.to_numeric(df_exportacao['quantidade_litros'], errors='coerce')
df_exportacao['valor_usd'] = pd.to_numeric(df_exportacao['valor_usd'], errors='coerce')

# Remover linhas com valores ausentes (NaN) em 'quantidade_litros' ou 'valor_usd'
df_clean = df_exportacao.dropna(subset=['quantidade_litros', 'valor_usd'])

# Verificar se há mais dados ausentes após limpeza
df_clean.isnull().sum()

id                   0
ano                  0
pais                 0
quantidade_litros    0
valor_usd            0
data_insercao        0
dtype: int64

In [4]:
# Codificar o país usando One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
pais_encoded = encoder.fit_transform(df_exportacao[['pais']])

# Criar um DataFrame com a codificação dos países
pais_encoded_df = pd.DataFrame(pais_encoded, columns=encoder.get_feature_names_out(['pais']))

# Adicionar as variáveis numéricas (ano, quantidade de litros, valor em USD) ao DataFrame
df_encoded = pd.concat([df_exportacao[['ano', 'quantidade_litros', 'valor_usd']], pais_encoded_df], axis=1)

# Normalizar as variáveis numéricas
scaler = StandardScaler()
df_encoded[['ano', 'quantidade_litros', 'valor_usd']] = scaler.fit_transform(df_encoded[['ano', 'quantidade_litros', 'valor_usd']])



In [5]:
print(df_encoded.columns)

Index(['ano', 'quantidade_litros', 'valor_usd', 'pais_Afeganistão',
       'pais_Alemanha, República Democrática', 'pais_Angola', 'pais_Anguilla',
       'pais_Antilhas Holandesas', 'pais_Antígua e Barbuda', 'pais_Argentina',
       ...
       'pais_Trinidade Tobago', 'pais_Tunísia', 'pais_Turquia', 'pais_Tuvalu',
       'pais_Uruguai', 'pais_Vanuatu', 'pais_Venezuela', 'pais_Vietnã',
       'pais_África do Sul', 'pais_Áustria'],
      dtype='object', length=141)


In [6]:
# Definir as variáveis de entrada (X) e as variáveis de saída (y)
X = df_encoded.drop(columns=['quantidade_litros', 'valor_usd'])  # Remover as colunas de saída
y = df_encoded['valor_usd']  # Colunas de saída


In [7]:

# Dividir os dados em conjunto de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preencher valores NaN com a média das colunas
X_train = X_train.fillna(X_train.mean())
y_train = y_train.fillna(y_train.mean())
X_test = X_test.fillna(X_test.mean())
y_test = y_test.fillna(y_test.mean())


# Exibir as dimensões dos conjuntos de treino e teste
print(f"Dimensões do conjunto de treino X: {X_train.shape}")
print(f"Dimensões do conjunto de teste X: {X_test.shape}")
print(f"Dimensões do conjunto de treino y: {y_train.shape}")
print(f"Dimensões do conjunto de teste y: {y_test.shape}")


Dimensões do conjunto de treino X: (1104, 139)
Dimensões do conjunto de teste X: (276, 139)
Dimensões do conjunto de treino y: (1104,)
Dimensões do conjunto de teste y: (276,)


## MODEL TRAINING

#### TRAINING (LINEAR REGRESSION)

In [ ]:
# # Criar o modelo de Random Forest
# model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# # Treinar o modelo com os dados de treino
# model_rf.fit(X_train, y_train)

# Aplicar o SMOTE para regressão
smote = SMOTER(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


# # Criar o modelo de Random Forest para múltiplas saídas
# model_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

# # Treinar o modelo com as variáveis de entrada (X) e saída (y)
# model_rf.fit(X_train, y_train)

# Exibir os coeficientes do modelo treinado
print("Modelo treinado com Random Forest - MultiOutputRegressor")


Modelo treinado com Random Forest - MultiOutputRegressor


#### EVALUATE MODEL

In [9]:
# Prever a quantidade de litros e o valor em USD com os dados de teste
y_pred = model_rf.predict(X_test)

# Calcular o erro quadrático médio (MSE) para avaliar o desempenho
mse = mean_squared_error(y_test, y_pred)

# Exibir o erro quadrático médio
print(f"Erro Quadrático Médio (MSE): {mse}")


Erro Quadrático Médio (MSE): 0.12767177530457266


In [10]:
print("Colunas usadas no treinamento:", X_train.columns.tolist())

Colunas usadas no treinamento: ['ano', 'pais_Afeganistão', 'pais_Alemanha, República Democrática', 'pais_Angola', 'pais_Anguilla', 'pais_Antilhas Holandesas', 'pais_Antígua e Barbuda', 'pais_Argentina', 'pais_Aruba', 'pais_Arábia Saudita', 'pais_Austrália', 'pais_Bahamas', 'pais_Bangladesh', 'pais_Barbados', 'pais_Barein', 'pais_Belice', 'pais_Benin', 'pais_Bermudas', 'pais_Bolívia', 'pais_Brasil', 'pais_Bulgária', 'pais_Bélgica', 'pais_Bósnia-Herzegovina', 'pais_Cabo Verde', 'pais_Camarões', 'pais_Canadá', 'pais_Catar', 'pais_Cayman, Ilhas', 'pais_Chile', 'pais_China', 'pais_Chipre', 'pais_Cingapura', 'pais_Cocos (Keeling), Ilhas', 'pais_Colômbia', 'pais_Comores', 'pais_Congo', 'pais_Coreia, Republica Sul', 'pais_Costa Rica', 'pais_Costa do Marfim', 'pais_Croácia', 'pais_Cuba', 'pais_Curaçao', 'pais_Dinamarca', 'pais_Dominica', 'pais_El Salvador', 'pais_Emirados Arabes Unidos', 'pais_Equador', 'pais_Eslovaca, Republica', 'pais_Espanha', 'pais_Estados Unidos', 'pais_Estônia', 'pais_Fil

### PREDICT

In [11]:
# Exemplo de previsão para o ano de 2024 e país "Brasil"
novo_ano = 2025
novo_pais = 'Brasil'

# Criar um DataFrame com o ano e a codificação do país
novo_pais_encoded = encoder.transform([[novo_pais]])
entrada = pd.DataFrame([list([novo_ano]) + list(novo_pais_encoded[0])], columns=X.columns)

# Fazer a previsão de quantidade de litros e valor em USD para o novo país e ano
valor_previsto = model_rf.predict(entrada)[0]

# Exibir os resultados da previsão
print(f"Previsão para o {novo_pais} em {novo_ano}:")
#print(f"Quantidade de Litros: {quantidade_prevista}")
print(f"Valor em USD: {valor_previsto}")


Previsão para o Brasil em 2025:
Valor em USD: -0.09832292161912871


/home/alberone/projects/fiap/ml-engineering/ml-tech-fiap-vitivinicultura-api/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


### SAVE MODEL

In [12]:
# Salvar o modelo treinado em um arquivo
joblib.dump(model_rf, 'modelo_exportacao.pkl')
# Salvar o encoder (caso necessário)
joblib.dump(encoder, 'encoder_exportacao.pkl')

['encoder_exportacao.pkl']